# VDS 1022/i Oscilloscope

To install :

Help:

In [ ]:
from vds1022 import *
help(vds1022)
help(vds1022.Frames)
help(vds1022.Frame)

Note that the first call of `VDS1022()` will take a few seconds since it has to load the FPGA firmware.
The device will remain connected unless `.dispose()` is called or if the kernel is restarted. Interrupting the kernel (`I`,`I`) will not terminate the connection.

### Examples :
1. [Live plotting](#1.-Live-plotting)
2. [Acquisition](#2.-Acquisition)
3. [Data logging](#3.-Data-logging)
4. [Continuous sampling](#4.-Continuous-sampling)
5. [FFT Spectrum Analysis](#5.-FFT-Spectrum-Analysis)
6. [Decoding](#6.-Decoding)

## 1. Live plotting


Plot the signals and refresh every 200ms

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('20ms')
dev.set_channel(CH1, range='20v', coupling=DC, offset=0, probe='x10')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2.5v', sweep=AUTO)
dev.plot();

In [ ]:
dev.stop();

## 2. Acquisition
Pull the buffer of the device once triggered and plot.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=AC, offset=0, probe='x10')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, position=0.5, level='2.5v', sweep=ONCE)
frames = dev.pull()
frames.plot()

Generate descriptive statistics.

In [ ]:
frames.describe()

Custom measures.

In [ ]:
(
    'CH1 Vmax: %sV'  % frames.ch1.max(),
    'CH1 Vavg: %sV' % frames.ch1.mean(),
    'CH1 Vrms: %sV'  % frames.ch1.rms()
)

Convert to [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html#dataframe)

In [ ]:
frames.to_dataframe()

## 3. Data logging
This section provides examples to measure a signal at a defined interval.

Measure and plot the RMS voltage.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x1')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x1')
dev.stream(freq=1).agg('rms').plot();

Measure and print the RMS voltage.

In [ ]:
from vds1022 import *

dev = VDS1022(debug=False)
dev.set_timerange('5ms')
dev.set_channel(CH1, range='10v', coupling=DC, offset=0, probe='x1')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x1')

for frames in dev.pull_iter():
    t = round(frames.time, 1)
    v = frames.ch1.rms()
    print('Time:%s  CH1:%sv     ' % (t, v), end='\r')

## 4. Continuous sampling

This section provides examples to aquire and plot continuous samples without interruption for a defined duration or on a trigger event.  
While this device can acquire sampling frames at 100Ms/s, the maximum continuous sampling rate is around 100Ks/s (20ms timerange).  
Since the amount of data can be quite consequent, the plotting has to go through rasterization.

Read continuous samples for a defined time

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022()
dev.set_sampling_rate(40e3)
dev.set_channel(CH1, DC, range='10v', offset=-0.4, probe='x10')
dev.set_channel(CH2, DC, range='10v', offset=-0.4, probe='x10')

df = dev.read('2s').to_dataframe()
df.hvplot(rasterize=True).opts(width=700, ylim=dev.ylim(), colorbar=False)

Read continuous samples on a trigger

In [ ]:
from vds1022 import *
import hvplot.pandas

dev = VDS1022()
dev.set_sampling_rate(40e3)
dev.set_channel(CH1, DC, range='10v', offset=0, probe='x10')
dev.set_channel(CH2, DC, range='10v', offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, level='2v', alternate=False, sweep=ONCE)

df = dev.read('500ms', pre='50ms').to_dataframe()
df.hvplot(rasterize=True).opts(width=800, ylim=dev.ylim(), colorbar=False)

## 5. FFT Spectrum Analysis

In [ ]:
from vds1022 import *

dev = VDS1022(debug=0)
dev.set_timerange('40ms')
dev.set_channel(CH1, range='10v', coupling=AC, offset=0, probe='x10')
dev.set_trigger(CH1, EDGE, RISE, position=0.5, level='2v', sweep=AUTO)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# acquire samples and compute the discrete Fourier Transform
frame = dev.pull().ch1
xf, yf = frame.fft()

# plot
plt.rcParams['axes.grid'] = True
fig, axs = plt.subplots(3, 1, figsize=(12, 3 * 2.3))
axs[0].set_xlabel('Time [ms]')
axs[0].set_ylabel('Amplitude [V]')
axs[0].plot(frame.x() * 1000, frame.y())
axs[1].set_xlabel('Frequency [kHz]')
axs[1].set_ylabel('Magnitude [Vrms]')
axs[1].plot(xf / 1000, yf)
axs[2].set_xlabel('Frequency [kHz]')
axs[2].set_ylabel('Magnitude [dBV]')
axs[2].plot(xf / 1000, 20 * np.log10(yf))
axs[2].yaxis.set_ticks(np.arange(-100, 30, 20));
plt.tight_layout()

## 6. Decoding I2C

In [ ]:
from vds1022 import *

dev = VDS1022()
dev.set_sampling_rate(100000)
dev.set_channel(CH1, range='10v', coupling=DC, offset=-0.1, probe='x1')
dev.set_channel(CH2, range='10v', coupling=DC, offset=0, probe='x1')
dev.set_trigger(CH1, EDGE, FALL, position=0.1, level='1.2v', sweep=ONCE)
frames = dev.pull()
frames.plot()

for msg in frames.decode_i2c():
    print(msg, msg.data)